# Develop Algorithm to Match Lists on Semantic Similarity

In [28]:
import os
import random
import re
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist, pdist, squareform
import spacy
from tqdm import tqdm

In [29]:
PROJ = Path(os.path.realpath("."))
ROOT = PROJ.parent
DATA = ROOT / "data"

In [30]:
# Prepare spacy
# python -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg")

# Read data

In [31]:
# Classification A
clas_df_a = pd.read_excel(DATA / "raw/Categories 2019_2018.xlsx")
clas_df_a = clas_df_a[["MAIN_TAXABLE_ACTIVITY 2018", "MAIN_TAXABLE_ACTIVITY 2019"]]
# Create one unique classification
clas_df_a = pd.DataFrame(
    clas_df_a["MAIN_TAXABLE_ACTIVITY 2018"]
    .append(clas_df_a["MAIN_TAXABLE_ACTIVITY 2018"])
    .unique(),
    columns=["clas_a_title"],
)
# Remove text in parantheses
clas_df_a["clas_a_title"] = clas_df_a["clas_a_title"].str.replace(r"\(.*\)", "").str.strip()
# Export
clas_df_a.head()

,clas_a_title
0,Business Support Services
1,Fishing
2,Agriculture Employment
3,Animal production
4,Support activities of Mining


In [32]:
# Classification B
sheets = ["HS2", "HS4", "NAICS2", "NAICS4"]

clas_df_b = pd.DataFrame()
for sheet in sheets:
    clas_df_b_sheet = pd.read_excel(DATA / "raw/codes.xlsx", sheet_name=sheet, dtype=str)
    clas_df_b_sheet.columns = ["clas_b_code", "clas_b_title"]
    clas_df_b_sheet["clas_b_type"] = sheet
    clas_df_b = clas_df_b.append(clas_df_b_sheet)
clas_df_b.head()

,clas_b_code,clas_b_title,clas_b_type
0,1,Live animals,HS2
1,2,Meat,HS2
2,3,Fish,HS2
3,4,Diary products,HS2
4,5,Animal products,HS2


# Pre-process data

In [37]:
def process_raw_classification(clas_df, titlecol, codecol=None):
    # Prepare unique classification df with a code
    clas_df = clas_df.drop_duplicates().copy()
    # Remove nulls
    clas_df = clas_df.dropna(subset=[titlecol])
    clas_df = clas_df[clas_df[titlecol].str.strip() != ""]
    # Create ID if there's none
    if codecol is None:
        clas_df["codecol"] = [f"a{str(x)}" for x in range(len(clas_df))]
        codecol = "codecol"
    # Error if codecol is duplicated
    if clas_df[codecol].duplicated().sum() > 0:
        raise ValueError("Code column is duplicated.")
    return clas_df

In [38]:
# Pre-process dfs
clas_df_a = process_raw_classification(clas_df_a, "clas_a_title")
clas_df_b = process_raw_classification(clas_df_b, "clas_b_title")

In [ ]:
# Get embeddings from spacy
